In [2]:
# SEO环境： schedule
import os
import warnings
warnings.filterwarnings('ignore')
import requests
import pandas as pd
import numpy as np
import jsonpath
import json
import time
import datetime
import xlwings as xw
import telebot
import hmac, base64, struct, hashlib
import math
from openpyxl import Workbook, load_workbook
from openpyxl import formatting, styles
from openpyxl.styles import Color, PatternFill, Font, Border
from PIL import ImageGrab
import pyperclip

pd.set_option('display.max_colwidth', None) #显示单元格完整信息
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

day = -1
pages_user = 150
pages_fircharge = 60
pages_fircharge_two = 100
pages_trade = 150

url_trade ='http://fundmng.bsportsadmin.com/api/manage/data/balance/record/list'
url_fircharge = 'http://fundmng.bsportsadmin.com/api/manage/data/detail/firstRecharge'
url_user = 'http://fundmng.bsportsadmin.com/api/manage/user/maintain/user/list'
url_huiyuan = 'http://fundmng.bsportsadmin.com/api/manage/data/loss/user/manage/list'  #会员流失

daili = pd.read_excel(r'C:\Users\User\Desktop\SEO\SEO提单数据\1011\代理线.xlsx')
# 第一次获取token
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'48',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-uid=690; admin-token=db76bebda5274c80adaadd40bd794f24',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/login',
    'Sign':'2bc4c378817f47731f0adf450a627d19',
    'Some':'header',
    'Systemid':"",
    'Timestamp':'1692415901000',
    'Token':'-1',
    'Uid':'-1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code


# 采集会员流失统计表
# huiyuan = pd.read_csv(r'C:\Users\User\Desktop\SEO\SEO提单数据\1011\流失会员详情.csv',encoding='gbk')
def huiyuan_fun(start_page,end_page):
    dic_huiyuan = {'会员账号':[],'代理':[],'vip等级':[],'首存时间':[]}
    for page in range(start_page,end_page+1):
        # 获取页码数量
        print(f'第{page}页。。。')
        data = {
            'page': page,
            'size': 500,
            'vipLevel': 0,
            'regStartTime': 1601481600000,
            'regEndTime': 1798854399999,
        }
        response = session.post(url=url_huiyuan,data=data,headers=header)
        response.encoding='utf8'
        obj = json.loads(response.text)

        for i in obj['data']['dataList']:
            dic_huiyuan['会员账号'].append(i['userName'])
            dic_huiyuan['代理'].append(i['parentName'])
            dic_huiyuan['vip等级'].append(i['vipLevel'])
            dic_huiyuan['首存时间'].append(i['lastRechargeDate'])

    huiyuan = pd.DataFrame(dic_huiyuan)
    print('huiyuan:',huiyuan.shape)
    return huiyuan
#获取原始token
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']

header = {
    'Device_id':'1.0',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/statistics/member-loss',
    'Sign':'6f518a02e3479ecaaf4ec58b3e5b3878',
    'Timestamp':'1697073050000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Version':'1.0'
}
# 总条数
data_init = {
    'page': 1,
    'size': 20,
    'vipLevel': 0,
    'regStartTime': 1601481600000,
    'regEndTime': 1798854399999,
}
session = requests.session()
response = session.post(url=url_huiyuan,data=data_init,headers=header)
obj_init = json.loads(response.text)
n_data = obj_init['data']['total']
print('总条数：',n_data)
pages = math.ceil(956377/500)
print('总页码：',pages)

#运行会员流失
huiyuan = huiyuan_fun(1,2)
huiyuan.head()

总条数： 957439
总页码： 1913
第1页。。。
第2页。。。
huiyuan: (1000, 4)


,会员账号,代理,vip等级,首存时间
0,proxy_thomas1101,admin,VIP0,--
1,windy2,admin,VIP0,--
2,ashin1000,admin,VIP0,--
3,ZZtaeri01,admin,VIP0,--
4,zzcyndi001,admin,VIP0,--


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# 会员流失 ，第一次获取token
import requests
import pandas as pd
import numpy as np
import jsonpath
import json
import time
import datetime
import xlwings as xw
import telebot
import hmac, base64, struct, hashlib
from openpyxl import Workbook, load_workbook
from openpyxl import formatting, styles
from openpyxl.styles import Color, PatternFill, Font, Border
from PIL import ImageGrab
import pyperclip
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Accept':'application/json, text/plain, */*',
    # 'Accept-Encoding':'gzip, deflate',
    'Accept-Language':'zh-CN,zh;q=0.9',
    'Connection':'keep-alive',
    'Content-Length':'48',
    'Content-Type':'application/x-www-form-urlencoded',
    'Cookie':'admin-uid=690; admin-token=db76bebda5274c80adaadd40bd794f24',
    'Device_id':'1.0',
    'Gl_version':'2.0',
    'Host':'fundmng.bsportsadmin.com',
    'Language':'zh_CN',
    'Origin':'http://fundmng.bsportsadmin.com',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/login',
    'Sign':'2bc4c378817f47731f0adf450a627d19',
    'Some':'header',
    'Systemid':"",
    'Timestamp':'1692415901000',
    'Token':'-1',
    'Uid':'-1',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code

# 采取token

data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']
# 采集会员流失记录
session = requests.session()
url_huiyuan = 'http://fundmng.bsportsadmin.com/api/manage/data/loss/user/manage/list'
header = {
    'Device_id':'1.0',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/statistics/member-loss',
    'Sign':'6f518a02e3479ecaaf4ec58b3e5b3878',
    'Timestamp':'1697073050000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Version':'1.0'
}
# 获取页码数量
data = {
    'page': 1,
    'size': 500,
    'vipLevel': 0,
    'regStartTime': 1601481600000,
    'regEndTime': 1798854399999,
}
response = session.post(url=url_huiyuan,data=data,headers=header)
response.text

'{"code":1,"data":{"dataList":[{"amountByCoin":{},"appLogin":"否","bank":"否","bettingAmount":0,"coin":"","firstTime":"--","lastBettingDaysTime":"614","lastLoginDate":"2022-02-06 01:18:25","lastLoginDaysTime":"613","lastRechargeDate":"--","lastRechargeDaysTime":"614","lastSuccessLoginDate":1644081505109,"lastTryLoginDate":"","loginIpAddr":"161.49.183.18","logoutDate":1644081685479,"parentId":1,"parentName":"admin","rechargeAmount":0,"regTime":"2022-02-05 23:54:15","remark":"","sameDevice":"否","telephone":"","uid":3,"userLevel":"禁止存款层级","userName":"proxy_thomas1101","userStatus":1,"userStatusName":"完全锁定","vipLevel":"VIP0","winAmount":0},{"amountByCoin":{},"appLogin":"否","bank":"否","bettingAmount":0,"coin":"","firstTime":"--","lastBettingDaysTime":"607","lastLoginDate":"2022-02-12 17:22:22","lastLoginDaysTime":"607","lastRechargeDate":"--","lastRechargeDaysTime":"613","lastSuccessLoginDate":1644657742973,"lastTryLoginDate":"","loginIpAddr":"112.206.103.76","logoutDate":1644741693867,"paren

In [12]:
obj = json.loads(response.text)
obj['data']['total']

956737

In [24]:
#计算页码
1913//10

191

In [16]:
import math
math.ceil(956377/500)

1913

In [22]:
pd.cut(range(1,math.ceil(956377/500)),bins=10,right=True,precision=0)

[(-1.0, 192.0], (-1.0, 192.0], (-1.0, 192.0], (-1.0, 192.0], (-1.0, 192.0], ..., (1721.0, 1912.0], (1721.0, 1912.0], (1721.0, 1912.0], (1721.0, 1912.0], (1721.0, 1912.0]]
Length: 1912
Categories (10, interval[float64, right]): [(-1.0, 192.0] < (192.0, 383.0] < (383.0, 574.0] < (574.0, 765.0] ... (1148.0, 1339.0] < (1339.0, 1530.0] < (1530.0, 1721.0] < (1721.0, 1912.0]]

In [9]:
import math
page_list = [

]
for i in range(0,math.ceil(956377/500),1913//10):
    page_list.append(i)
page_list

[0, 191, 382, 573, 764, 955, 1146, 1337, 1528, 1719, 1910]

In [12]:
math.ceil(956377/500)

1913

In [6]:
page_list = page_list.append(math.ceil(956377/500))
page_list

In [52]:
for i in range(11):
    print(page_list[i]+1,page_list[i+1])

1 191
192 382
383 573
574 764
765 955
956 1146
1147 1337
1338 1528
1529 1719
1720 1910
1911 1913


In [11]:
[(page_list[i]+1,page_list[i+1]) for i in range(10) ]

[(1, 191),
 (192, 382),
 (383, 573),
 (574, 764),
 (765, 955),
 (956, 1146),
 (1147, 1337),
 (1338, 1528),
 (1529, 1719),
 (1720, 1910)]

In [34]:
page_list[len(page_list)-1]

1913

In [7]:
session.get(url=url_huiyuan,data=data,headers=header).text

'{"code":405,"message":"登录失效"}'

In [ ]:
huiyuan = pd.DataFrame()
def huiyuan_fun(start_page,end_page):
    dic_huiyuan = {'会员账号':[],'代理':[],'vip等级':[],'首存时间':[]}
    for page in range(start_page,end_page+1):
        # 获取页码数量
        data = {
            'page': page,
            'size': 500,
            'vipLevel': 0,
            'regStartTime': 1601481600000,
            'regEndTime': 1798854399999,
        }
        response = session.post(url=url_huiyuan,data=data,headers=header)
        response.encoding='utf8'
        obj = json.loads(response.text)
        for i in obj['data']['datalist']:
            dic_huiyuan['会员账号'].append(i['userName'])
            dic_huiyuan['代理'].append(i['parentName'])
            dic_huiyuan['vip等级'].append(i['vipLevel'])
            dic_huiyuan['首存时间'].append(i['lastRechargeDate'])

df = pd.DataFrame(columns=['会员账号', '代理', 'vip等级', '首存时间'])

In [81]:
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']
# 采集会员流失记录

url_huiyuan = 'http://fundmng.bsportsadmin.com/api/manage/data/loss/user/manage/list'
header = {
    'Device_id':'1.0',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/statistics/member-loss',
    'Sign':'6f518a02e3479ecaaf4ec58b3e5b3878',
    'Timestamp':'1697073050000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Version':'1.0'
}
import multiprocessing
def huiyuan_q_fun(start_page,end_page,q):
    session = requests.session()
    dic_huiyuan = {'会员账号':[],'代理':[],'vip等级':[],'首存时间':[]}
    for page in range(start_page,end_page+1):
        # 获取页码数量
        data = {
            'page': page,
            'size': 500,
            'vipLevel': 0,
            'regStartTime': 1601481600000,
            'regEndTime': 1798854399999,
        }
        response = session.post(url=url_huiyuan,data=data,headers=header)
        response.encoding='utf8'
        obj = json.loads(response.text)
        print(response.text)
        for i in obj['data']['dataList']:
            dic_huiyuan['会员账号'].append(i['userName'])
            dic_huiyuan['代理'].append(i['parentName'])
            dic_huiyuan['vip等级'].append(i['vipLevel'])
            dic_huiyuan['首存时间'].append(i['lastRechargeDate'])

    q.put({start_page:dic_huiyuan},timeout=100)
    print(pd.DataFrame(dic_huiyuan).shape)

q = multiprocessing.Queue()


jobs = []
for i in range(11):
    p = multiprocessing.Process(target=huiyuan_q_fun, args=(page_list[i]+1,page_list[i+1], q))
    jobs.append(p)
    p.start()

for p in jobs:
    p.join()

while not q.empty():
    print('队列不为空')
    huiyuan_df = pd.DataFrame(q.get(timeout=100))
    huiyuan_df=huiyuan_df.append(huiyuan_df)

huiyuan_df

""


In [ ]:
with multiprocessing.Pool(processes=10) as pool:
    for i in range(11):
       pool.starmap(huiyuan_q_fun, (page_list[i]+1,page_list[i+1], q))
print(q.maxsize)

In [69]:
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
}
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']
# 采集会员流失记录
session = requests.session()
url_huiyuan = 'http://fundmng.bsportsadmin.com/api/manage/data/loss/user/manage/list'
header = {
    'Device_id':'1.0',
    'Os_type':'0',
    'Referer':'http://fundmng.bsportsadmin.com/system/statistics/member-loss',
    'Sign':'6f518a02e3479ecaaf4ec58b3e5b3878',
    'Timestamp':'1697073050000',
    'Token':token,
    'Uid':'690',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'Version':'1.0'
}
def huiyuan_q2_fun(start_page,end_page):
    session = requests.session()
    dic_huiyuan = {'会员账号':[],'代理':[],'vip等级':[],'首存时间':[]}
    for page in range(start_page,end_page+1):
        # 获取页码数量
        data = {
            'page': page,
            'size': 500,
            'vipLevel': 0,
            'regStartTime': 1601481600000,
            'regEndTime': 1798854399999,
        }
        response = session.post(url=url_huiyuan,data=data,headers=header)
        response.encoding='utf8'
        print(response.text)
        obj = json.loads(response.text)
        for i in obj['data']['dataList']:
            dic_huiyuan['会员账号'].append(i['userName'])
            dic_huiyuan['代理'].append(i['parentName'])
            dic_huiyuan['vip等级'].append(i['vipLevel'])
            dic_huiyuan['首存时间'].append(i['lastRechargeDate'])

    return pd.DataFrame(dic_huiyuan).shape
huiyuan_q2_fun(1,3)

{"code":1,"data":{"dataList":[{"amountByCoin":{},"appLogin":"否","bank":"否","bettingAmount":0,"coin":"","firstTime":"--","lastBettingDaysTime":"614","lastLoginDate":"2022-02-06 01:18:25","lastLoginDaysTime":"613","lastRechargeDate":"--","lastRechargeDaysTime":"614","lastSuccessLoginDate":1644081505109,"lastTryLoginDate":"","loginIpAddr":"161.49.183.18","logoutDate":1644081685479,"parentId":1,"parentName":"admin","rechargeAmount":0,"regTime":"2022-02-05 23:54:15","remark":"","sameDevice":"否","telephone":"","uid":3,"userLevel":"禁止存款层级","userName":"proxy_thomas1101","userStatus":1,"userStatusName":"完全锁定","vipLevel":"VIP0","winAmount":0},{"amountByCoin":{},"appLogin":"否","bank":"否","bettingAmount":0,"coin":"","firstTime":"--","lastBettingDaysTime":"607","lastLoginDate":"2022-02-12 17:22:22","lastLoginDaysTime":"607","lastRechargeDate":"--","lastRechargeDaysTime":"613","lastSuccessLoginDate":1644657742973,"lastTryLoginDate":"","loginIpAddr":"112.206.103.76","logoutDate":1644741693867,"parent

(1500, 4)

""


In [ ]:
import random
import multiprocessing

def worker(id, q):
    try:
        print('process {} start'.format(id))
        x = id / random.randint(-1, 3)
        q.put({id: {"result": int(x)}})
    except Exception as e:
        q.put({id: {"error": str(e)}})

if __name__ == "__main__":
    q = multiprocessing.Queue()
    jobs = []
    for i in range(1, 11):
        p = multiprocessing.Process(target=worker, args=(i, q))
        jobs.append(p)
        p.start()
    for p in jobs:
        p.join()
    result_list = []
    while not q.empty():
        result_list.append(q.get())
    print(result_list)


In [ ]:
import multiprocessing
with multiprocessing.Pool(processes=10) as pool:
    pool.starmap(huiyuan_fun,[(page_list[i]+1,page_list[i+1]) for i in range(11) ])